In [1]:
import pandas as pd
import os
import numpy as np
import datetime as dt

from os import listdir
from os.path import isfile, join
from os.path import getmtime
import zipfile

from zebra import remov_duplicates, human_format
from directories import *

In [2]:
idx = pd.IndexSlice

In [3]:
pd.set_option('display.max_rows', 200)

# Sources

## Parametriche

In [4]:
par = {}
for file in parametriche_files:
    file_name=file.split(".")[0]
    par[f"{file_name}"] = pd.read_excel(parametriche+"\\"+file)

In [5]:
cod_prod = pd.DataFrame(par["Classificazione_codice_prodotto"]).loc[:,['MDM', 'GTM BDG', 'Solution','Sub solution',"Descrizione prodotto","Codice PRODOTTO"]].assign(**{"Codice PRODOTTO": lambda x: x["Codice PRODOTTO"].astype(str)})

sales_director = pd.DataFrame(par["Sales_director"]).loc[:,["Sales Director", "Area territoriale", "Rete"]].assign(**{"Sales Director": lambda x: x["Sales Director"].str.lower()})

esc_fatture = pd.DataFrame(par["Esclusione_fatture"]).assign(**{"Numero Fattura": lambda x: x["Numero Fattura"].astype(str)})

esc_ordini = pd.DataFrame(par["Esclusione_ordini"]).assign(**{"Numero Ordine": lambda x: x["Numero Ordine"].astype(str)})

agente_ordine = pd.DataFrame(par["Agente_ordine"])

li_prodotti = pd.DataFrame(par["Linea_prodotti"])

## Parco clienti da Licenze e abbonamenti

In [6]:
zip_file = zip_folder+r"\Abbonamenti attivi indiretta WK Info.ZIP"

try:
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(abbonamenti)
except:
    print("Invalid file")

In [7]:
old_file = os.path.join(abbonamenti, "ZQ_ABBO_EDIT_VAL_RINNOVO_00000.xls")
new_file = os.path.join(abbonamenti, "Clienti_Indiretta WK Info.xls")
os.replace(old_file, new_file)

In [8]:
zip_file = zip_folder+r"\Abbonamenti attivi diretta e indiretta.ZIP"

try:
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(abbonamenti)
except:
    print("Invalid file")

In [9]:
old_file = os.path.join(abbonamenti, "ZQ_ABBO_EDIT_VAL_RINNOVO_00000.xls")
new_file = os.path.join(abbonamenti, "Clienti_Diretta-Indiretta.xls")
os.replace(old_file, new_file)

In [10]:
zip_file = zip_folder+r"\Clienti merce da abbonamenti.ZIP"

try:
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(anagrafiche)
except:
    print("Invalid file")

In [11]:
old_file = os.path.join(anagrafiche, "ZQ_ABBO_EDIT_VAL_RINNOVO_00000.xls")
new_file = os.path.join(anagrafiche, "Lista_clienti_merce_da_abbonamenti.xls")
os.replace(old_file, new_file)

___

In [12]:
dfs = []
for file in abb_files:
    data = pd.read_excel(abbonamenti+"\\"+file, na_values="#", dtype={"Data inizio contratt":object, "Data Inizio Abb.":object, "Data Fine Abb.":object, "Cliente MERCE":str, "Cliente FATTURA":str, "Codice ABBONAMENTO":str})
    data["Tipo Licenza"] = file.split("_")[1].split(".")[0]
    data["Ultimo Agg."] = dt.date.fromtimestamp(getmtime(abbonamenti+"\\"+file))
    dfs.append(data)

cli = pd.concat(dfs, axis=0, ignore_index=True)
    
for col in cli.columns:
    if "Data" in col:
        
        cli[col] = cli[col].apply(lambda x: (float(x) - 25569) * 86400)
        cli[col] = pd.to_datetime(cli[col], unit="s")
    else:
        pass

## Parco clienti da Licenze e abbonamenti

In [13]:
cli_merce = pd.read_excel(anagrafiche+"\\"+"Lista_clienti_merce_da_abbonamenti.xlsb", usecols=['Cliente MERCE', 'Segmento Aggregato Cl.Merce', 'Provincia', 'Regione', 'Cliente Padre Merce'], dtype={"Cliente MERCE":str})

## All accounts da Salesforce

In [13]:
import win32com.client

File = win32com.client.Dispatch("Excel.Application")
Workbook = File.Workbooks.Open(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Salesforce\Data export\It_all_accounts.xlsb")
File.Visible = True
Workbook.RefreshAll()
File.CalculateUntilAsyncQueriesDone()
Workbook.Save()
Workbook.Close(SaveChanges=False)
File.Quit()

In [24]:
accounts_sf = pd.read_excel(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Salesforce\Data export\It_all_accounts.xlsb", sheet_name="Account")

In [ ]:
accounts_sf

## Filtri

In [14]:
stato_abb = (cli["Stato Esteso"].str.contains("ATTIVO")) & (cli["Stato Sintetico Contratto"].isin(["Attivo","Sospeso"]))

In [31]:
columns = ["Cliente MERCE","Segmento Aggregato Cl.Merce",'Canale di VENDITA',"Agente dell'ORDINE", 'RSM Agente', "Rete", "Area territoriale", 'Regione', 'Provincia', 'GTM BDG', 'Solution', 'Sub solution', "Descrizione prodotto", "Codice PRODOTTO", "Sistema","Fornitura (SW)","Valore PV New"]

#'Cliente MERCE', 'Segmento Aggregato Cl.Merce', 'Regione', 'Provincia','Canale di VENDITA', 'Cliente FATTURA', 'Numero Contratto Lic','Stato Sintetico Contratto', 'Data inizio contratt','Codice ABBONAMENTO', 'Unnamed: 10', 'Soluzione (SW)', 'Prodotto (SW)','Cluster (SW)', 'Sistema', 'Fornitura (SW)', "Agente dell'Abbonamento",'Sales Director', 'Stato Esteso', 'Data Inizio Abb.', 'Data Fine Abb.','Valore PV New', 'Tipo Licenza', 'Ultimo Agg.'
#'MDM', 'GTM BDG', 'Solution', 'Sub solution', 'Codice PRODOTTO'
#'Sales Director', 'Area territoriale', 'Rete'

## Creazione dataframe filtrato

In [32]:
def cli_prod_saldir_ag(df_):

  df = cli.loc[stato_abb, :].copy()
  df["Sales Director"] = df["Sales Director"].str.lower()
  df = df.merge(cli_merce, how="left", on="Cliente MERCE")\
    .merge(cod_prod, how="left", left_on="Codice ABBONAMENTO", right_on="Codice PRODOTTO")\
    .merge(sales_director, how="left", on="Sales Director")\
    .merge(agente_ordine.loc[:,['Codice agente', "Agente dell'ORDINE", 'RSM Agente']], how="left", left_on="Agente dell'Abbonamento", right_on="Codice agente")
    
  return df.loc[df["Rete"] == "Indiretta agenzie", columns]

#df["Solution"].isin(solutions)

In [33]:
clienti = cli.pipe(cli_prod_saldir_ag)

### Classificazione per Sorbello

In [85]:
selection = [
    (clienti["GTM BDG"] == "Arkon")|(clienti["GTM BDG"] == "SMART Skills")|(clienti["GTM BDG"] == "WD+FS"),
    (clienti["GTM BDG"] == "Genya Bilancio")|(clienti["GTM BDG"] == "Genya CFO")|(clienti["GTM BDG"] == "Genya Suite")|(clienti["GTM BDG"] == "Legacy Tax")|(clienti["GTM BDG"] == "Revisya"),
    (clienti["GTM BDG"] == "Arca"),
    (clienti["GTM BDG"] == "Legacy Paghe")]
choices = ["Invoicing & Archiving", "Tax", "ERP", "Payroll"]
clienti["Sorbello"] =np.select(selection, choices, "Altro")

In [86]:
clienti.groupby(["Sorbello"], dropna=False).agg({"Cliente MERCE":"nunique"}).rename(columns={"Cliente MERCE":"N.Clienti"})

,N.Clienti
Sorbello,
Altro,583
ERP,10789
Invoicing & Archiving,20063
Payroll,3535
Tax,20663


## Modifica valore Solution per comprendere uno o più valori della Sub Solution

In [35]:
clienti.loc[(clienti["Sub solution"] == "Tuttotel FE"),  "Solution"] = "Tuttotel FE"
clienti.loc[(clienti["Sub solution"] == "DIGY EC"),  "Solution"] = "DIGY EC"

## Creazione cluster prodotti

In [36]:
def concat_prodotti(df_):
    df = clienti.loc[:,["Solution", "Cliente MERCE"]].fillna("Senza attribuz.")
    df["Solution"] = pd.Categorical(df["Solution"], np.sort(df["Solution"].unique()))
    df.sort_values(by=["Solution"], inplace=True)
    df["Prodotti"] = df.groupby("Cliente MERCE")["Solution"].transform(lambda x: " + ".join(x)).apply(lambda x: remov_duplicates(x))
    return df["Prodotti"]

In [37]:
def pv_max(df_):
    df = clienti.loc[:,["Cliente MERCE", "Valore PV New"]]
    df["Valore PV cliente"] = df.groupby("Cliente MERCE")["Valore PV New"].transform('sum')
    return df["Valore PV cliente"]

In [38]:
clienti = clienti.assign(Prodotti = lambda df_: df_.pipe(concat_prodotti)).assign(**{"Valore PV cliente": lambda df_: df_.pipe(pv_max)})
#.assign(**{"Codice PRODOTTO": lambda x: x["Codice PRODOTTO"].astype(str)})

In [27]:
clienti.columns

Index(['Cliente MERCE', 'Segmento Aggregato Cl.Merce', 'Canale di VENDITA',
       'Agente dell'ORDINE', 'Rete', 'Area territoriale', 'Regione',
       'Provincia', 'GTM BDG', 'Solution', 'Sub solution',
       'Descrizione prodotto', 'Codice PRODOTTO', 'Sistema', 'Fornitura (SW)',
       'Valore PV New', 'Prodotti', 'Valore PV cliente'],
      dtype='object')

## Aggiunta della ragione sociale

In [39]:
clienti["Codice SAP"] = clienti["Cliente MERCE"].astype(str).apply(lambda x: "IT-" + x.zfill(10))
clienti = clienti.merge(accounts_sf, how="left", left_on="Codice SAP", right_on="AccountNumber")

## Ultima definizione colonne

In [40]:
clienti = clienti.loc[:,["Cliente MERCE", "Name", "Segmento Aggregato Cl.Merce", "Agente dell'ORDINE", 'RSM Agente', "Area territoriale", 'Regione', 'Provincia', "Fornitura (SW)", "Prodotti", "Sistema", "Solution","Sub solution", "Valore PV New", "Valore PV cliente"]].copy()
clienti.rename(columns={"Valore PV New":"Valore PV", "Agente dell'ORDINE":"Agente", "Name":"Account Name"}, inplace=True)

In [44]:
clienti.to_csv(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Analisi\xTemp\Clienti_agenzie_20230928.csv", index=False)

In [39]:
clienti_gr = clienti.groupby(["Cliente MERCE", "Account Name", "Agente", "Prodotti"], dropna=False).agg({"Valore PV":"sum"}).reset_index()

In [ ]:
with pd.ExcelWriter(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Analisi clienti\Clienti per area territoriale\Clienti diretta\EED e LA\Estrazione clienti LA\Estrazione clienti - 20230904.xlsx") as writer:
    clienti.to_excel(writer, sheet_name="Dettaglio per prod", index=False)
    clienti_gr.to_excel(writer, sheet_name="Raggr. per cli", index=False)

In [ ]:
clienti.loc[(clienti["Solution"] == "Arca Evolution")&(clienti["Sub solution"] == "Contabilità analitica"),:].to_excel(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Analisi clienti\Clienti per area territoriale\Clienti diretta\EED e LA\Abbonamenti attivi\Abbonamenti attivi - Contabilità analitica.xlsx", index=False)